In [3]:
from google.colab import files
uploaded = files.upload()

Saving dataset.zip to dataset.zip


In [6]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Datasets/Indian Food Image Dataset.zip"  # Update this path if filename is different
extract_path = "/content/dataset/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Extraction complete")


✅ Extraction complete


In [7]:
import os

for root, dirs, files in os.walk(extract_path):
    print("📂", root)
    for d in dirs:
        print("  📁", d)
    for f in files[:5]:  # just print a few files
        print("  📄", f)
    break  # stop after the first level


📂 /content/dataset/
  📁 train
  📁 dataset
  📁 test


In [5]:
!ls


dataset  drive	sample_data


In [8]:
inner_path = "/content/dataset/dataset"

for root, dirs, files in os.walk(inner_path):
    print("📂", root)
    for d in dirs:
        print("  📁", d)
    for f in files[:5]:
        print("  📄", f)
    break


📂 /content/dataset/dataset
  📁 train
  📁 test


In [9]:
import tensorflow as tf

# Set parameters
img_size = (224, 224)
batch_size = 32
val_split = 0.2

# Paths
train_path = "/content/dataset/dataset/train"
test_path = "/content/dataset/dataset/test"

# Load full training dataset first
full_train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    label_mode='categorical',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    seed=123
)

# Determine split sizes
train_batches = tf.data.experimental.cardinality(full_train_ds).numpy()
val_ds = full_train_ds.take(int(val_split * train_batches))
train_ds = full_train_ds.skip(int(val_split * train_batches))

# Load test set
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path,
    label_mode='categorical',
    image_size=img_size,
    batch_size=batch_size
)

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 3200 files belonging to 80 classes.
Found 800 files belonging to 80 classes.


In [ ]:
print(full_train_ds.class_names)
print(len(full_train_ds.class_names))  # should match num_classes in your final Dense layer

['adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi', 'aloo_shimla_mirch', 'aloo_tikki', 'anarsa', 'ariselu', 'bandar_laddu', 'basundi', 'bhatura', 'bhindi_masala', 'biryani', 'boondi', 'butter_chicken', 'chak_hao_kheer', 'cham_cham', 'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala', 'chicken_tikka', 'chicken_tikka_masala', 'chikki', 'daal_baati_churma', 'daal_puri', 'dal_makhani', 'dal_tadka', 'dharwad_pedha', 'doodhpak', 'double_ka_meetha', 'dum_aloo', 'gajar_ka_halwa', 'gavvalu', 'ghevar', 'gulab_jamun', 'imarti', 'jalebi', 'kachori', 'kadai_paneer', 'kadhi_pakoda', 'kajjikaya', 'kakinada_khaja', 'kalakand', 'karela_bharta', 'kofta', 'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha', 'lyangcha', 'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua', 'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan', 'navrattan_korma', 'palak_paneer', 'paneer_butter_masala', 'phirni', 'pithe', 'poha', 'poornalu', 'pootharekulu', 'qubani_ka_meetha', 'rabri', 'ras_malai', 'ras

In [ ]:
for images, labels in train_ds.take(1):
    print("Shape of labels:", labels.shape)
    print("Unique labels:", tf.unique(tf.argmax(labels, axis=1) if labels.shape[-1] > 1 else labels)[0])


Shape of labels: (32, 80)
Unique labels: tf.Tensor(
[66  8 34 23 39 10 64 74 27  0 60 79 61 59 30  9 24 15 38 21 62 53  2 49
 36], shape=(25,), dtype=int64)


In [10]:
# Data Augmentation Layer
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
], name="data_augmentation")


In [11]:
# # Custom CNN
# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(224, 224, 3)),
#     data_augmentation,
#     tf.keras.layers.Rescaling(1./255),

#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),

#     tf.keras.layers.Conv2D(64, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),

#     tf.keras.layers.Conv2D(128, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(80, activation='softmax')
# ])

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze base

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(80, activation='softmax')
])




9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [14]:
checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [15]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=200,  # start with 50, can go up later
    callbacks=[checkpoint, early_stop] #early stopping is removed
)

Epoch 1/200
79/80 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0355 - loss: 4.5160
Epoch 1: val_accuracy improved from -inf to 0.16094, saving model to best_model.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 28s 106ms/step - accuracy: 0.0359 - loss: 4.5099 - val_accuracy: 0.1609 - val_loss: 3.6370
Epoch 2/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.1786 - loss: 3.4348
Epoch 2: val_accuracy improved from 0.16094 to 0.34219, saving model to best_model.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 25s 71ms/step - accuracy: 0.1787 - loss: 3.4333 - val_accuracy: 0.3422 - val_loss: 2.8236
Epoch 3/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.2850 - loss: 2.8292
Epoch 3: val_accuracy improved from 0.34219 to 0.41406, saving model to best_model.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.2851 - loss: 2.8281 - val_accuracy: 0.4141 - val_loss: 2.4293
Epoch 4/200
79/80 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3643 - loss: 2.4140
Epoch 4: val_accuracy improved from 0.41406 to 0.45156, saving model to best_model.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.3644 - loss: 2.4133 - val_accuracy: 0.4516 - val_loss: 2.1631
Epoch 5/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4254 - loss: 2.1014
Epoch 5: val_accuracy improved from 0.45156 to 0.48438, saving model to best_model.h5


...

80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.6315 - loss: 1.2655 - val_accuracy: 0.5484 - val_loss: 1.7063
Epoch 13/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6796 - loss: 1.1568
Epoch 13: val_accuracy improved from 0.54844 to 0.56406, saving model to best_model.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - accuracy: 0.6793 - loss: 1.1572 - val_accuracy: 0.5641 - val_loss: 1.6532
Epoch 14/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6730 - loss: 1.0942
Epoch 14: val_accuracy improved from 0.56406 to 0.56563, saving model to best_model.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step - accuracy: 0.6728 - loss: 1.0946 - val_accuracy: 0.5656 - val_loss: 1.6750
Epoch 15/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6740 - loss: 1.0768
Epoch 15: val_accuracy improved from 0.56563 to 0.59062, saving model to best_model.h5


80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - accuracy: 0.6740 - loss: 1.0770 - val_accuracy: 0.5906 - val_loss: 1.6198
Epoch 16/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7137 - loss: 0.9755
Epoch 16: val_accuracy did not improve from 0.59062
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - accuracy: 0.7136 - loss: 0.9757 - val_accuracy: 0.5688 - val_loss: 1.6391
Epoch 17/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7326 - loss: 0.9521
Epoch 17: val_accuracy did not improve from 0.59062
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step - accuracy: 0.7324 - loss: 0.9525 - val_accuracy: 0.5703 - val_loss: 1.6529
Epoch 18/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7597 - loss: 0.8583
Epoch 18: val_accuracy did not improve from 0.59062
80/80 ━━━━━━━━━━━━━━━━━━━━ 11s 70ms/step - accuracy: 0.7596 - loss: 0.8585 - val_accuracy: 0.5609 - val_loss: 1.6673
Epoch 19/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.7490 - loss: 0.8329
Epoch 19: val_accuracy did no

80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.7332 - loss: 0.8469 - val_accuracy: 0.5922 - val_loss: 1.6283
Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 15.


In [ ]:
import os

def count_images_per_class(data_dir):
    class_counts = {}
    for class_name in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):
            image_count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
            class_counts[class_name] = image_count
    return class_counts

# Example usage
data_dir = train_path # Update this path as needed
counts = count_images_per_class(data_dir)

# Display
for class_name, count in sorted(counts.items(), key=lambda x: x[0]):
    print(f"{class_name}: {count} images")


adhirasam: 40 images
aloo_gobi: 40 images
aloo_matar: 40 images
aloo_methi: 40 images
aloo_shimla_mirch: 40 images
aloo_tikki: 40 images
anarsa: 40 images
ariselu: 40 images
bandar_laddu: 40 images
basundi: 40 images
bhatura: 40 images
bhindi_masala: 40 images
biryani: 40 images
boondi: 40 images
butter_chicken: 40 images
chak_hao_kheer: 40 images
cham_cham: 40 images
chana_masala: 40 images
chapati: 40 images
chhena_kheeri: 40 images
chicken_razala: 40 images
chicken_tikka: 40 images
chicken_tikka_masala: 40 images
chikki: 40 images
daal_baati_churma: 40 images
daal_puri: 40 images
dal_makhani: 40 images
dal_tadka: 40 images
dharwad_pedha: 40 images
doodhpak: 40 images
double_ka_meetha: 40 images
dum_aloo: 40 images
gajar_ka_halwa: 40 images
gavvalu: 40 images
ghevar: 40 images
gulab_jamun: 40 images
imarti: 40 images
jalebi: 40 images
kachori: 40 images
kadai_paneer: 40 images
kadhi_pakoda: 40 images
kajjikaya: 40 images
kakinada_khaja: 40 images
kalakand: 40 images
karela_bharta: 40

In [ ]:

# Example usage
data_dir = test_path # Update this path as needed
counts = count_images_per_class(data_dir)

# Display
for class_name, count in sorted(counts.items(), key=lambda x: x[0]):
    print(f"{class_name}: {count} images")


adhirasam: 10 images
aloo_gobi: 10 images
aloo_matar: 10 images
aloo_methi: 10 images
aloo_shimla_mirch: 10 images
aloo_tikki: 10 images
anarsa: 10 images
ariselu: 10 images
bandar_laddu: 10 images
basundi: 10 images
bhatura: 10 images
bhindi_masala: 10 images
biryani: 10 images
boondi: 10 images
butter_chicken: 10 images
chak_hao_kheer: 10 images
cham_cham: 10 images
chana_masala: 10 images
chapati: 10 images
chhena_kheeri: 10 images
chicken_razala: 10 images
chicken_tikka: 10 images
chicken_tikka_masala: 10 images
chikki: 10 images
daal_baati_churma: 10 images
daal_puri: 10 images
dal_makhani: 10 images
dal_tadka: 10 images
dharwad_pedha: 10 images
doodhpak: 10 images
double_ka_meetha: 10 images
dum_aloo: 10 images
gajar_ka_halwa: 10 images
gavvalu: 10 images
ghevar: 10 images
gulab_jamun: 10 images
imarti: 10 images
jalebi: 10 images
kachori: 10 images
kadai_paneer: 10 images
kadhi_pakoda: 10 images
kajjikaya: 10 images
kakinada_khaja: 10 images
kalakand: 10 images
karela_bharta: 10

In [18]:
import tensorflow as tf

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/dataset/test',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32)

test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

model.evaluate(test_ds)


Found 800 files belonging to 80 classes.
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.5143 - loss: 1.7173


[1.73209547996521, 0.5249999761581421]

In [20]:
base_model.trainable = True

# Freeze earlier layers – MobileNetV2 has 155 layers
for layer in base_model.layers[:100]:
    layer.trainable = False


In [21]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.RandomBrightness(0.2)
])


In [22]:

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze base

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(80, activation='softmax')
])

In [24]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [25]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)
checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,  # start with 50, can go up later
    callbacks=[checkpoint, early_stop] #early stopping is removed
)

Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0219 - loss: 4.7712
Epoch 1: val_accuracy improved from -inf to 0.02969, saving model to best_model.h5


TypeError: cannot pickle 'module' object

In [26]:
from tensorflow.keras import layers, Model

# Data Augmentation block
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.3),
    layers.RandomBrightness(0.2),
], name="data_augmentation")

# Base model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True

# Freeze first 100 layers
for layer in base_model.layers[:100]:
    layer.trainable = False

# Input layer
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(80, activation='softmax')(x)

model = Model(inputs, outputs)


In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[checkpoint]
)


Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0199 - loss: 4.6286
Epoch 1: val_accuracy improved from 0.02969 to 0.05469, saving model to best_model.h5


TypeError: cannot pickle 'module' object